# PyQIR to Squin Conversion

This notebook demonstrates two different approaches to converting quantum circuits to squin kernels:

1. **qbraid-qir squin implementation**: Converts PyQIR modules directly to squin kernels
2. **bloqade cirq lowering**: Converts Cirq circuits to squin kernels (for comparison)

Both approaches are shown using a GHZ (Greenberger-Horne-Zeilinger) state preparation circuit.


In [ ]:
# Import required libraries
from pyqir import BasicQisBuilder, SimpleModule
from qbraid_qir.squin import load
import cirq
import bloqade.cirq_utils.lowering as lowering_module
from bloqade.pyqrack import StackMemorySimulator

## Part 1: PyQIR → Squin (qbraid-qir implementation)

This section demonstrates the qbraid-qir squin implementation, which converts PyQIR modules directly to squin kernels.


In [ ]:
# Create a GHZ state preparation circuit using PyQIR
n = 4
mod = SimpleModule("ghz_n", num_qubits=n, num_results=n)
qis = BasicQisBuilder(mod.builder)

qis.h(mod.qubits[0])
for i in range(n - 1):
    qis.cx(mod.qubits[i], mod.qubits[i + 1])

In [7]:
# Convert PyQIR module to squin kernel using qbraid-qir
squin_mod = load(mod.ir(), kernel_name="ghz_n")

print("\nSquin Kernel (from qbraid-qir):")
print("=" * 60)
squin_mod.print()


Squin Kernel (from qbraid-qir):
func.func @ghz_n() -> !py.NoneType {
  ^0(%ghz_n_self):
  │ %0 = func.invoke new() : !py.Qubit maybe_pure=False
  │ %1 = func.invoke new() : !py.Qubit maybe_pure=False
  │ %2 = func.invoke new() : !py.Qubit maybe_pure=False
  │ %3 = func.invoke new() : !py.Qubit maybe_pure=False
  │ %4 = func.invoke h(%0) : !py.NoneType maybe_pure=False
  │ %5 = func.invoke cx(%0, %1) : !py.NoneType maybe_pure=False
  │ %6 = func.invoke cx(%1, %2) : !py.NoneType maybe_pure=False
  │ %7 = func.invoke cx(%2, %3) : !py.NoneType maybe_pure=False
  │ %8 = func.const.none() : !py.NoneType
  │      func.return %8
} // func.func ghz_n


## Part 2: Cirq → Squin (bloqade implementation)

This section demonstrates the bloqade implementation, which converts Cirq circuits to squin kernels. This allows us to compare the squin kernels generated from different source formats.


In [8]:
# Define the same GHZ circuit using Cirq
def ghz_circuit(n):
    qubits = cirq.LineQubit.range(n)
    circuit = cirq.Circuit()
    circuit.append(cirq.H(qubits[0]))
    for i in range(n - 1):
        circuit.append(cirq.CNOT(qubits[i], qubits[i + 1]))

    return circuit

In [9]:
# Create the same GHZ circuit using Cirq
circuit = ghz_circuit(4)
print("Cirq Circuit:")
print("=" * 60)
print(circuit)
print()

Cirq Circuit:
0: ───H───@───────────
          │
1: ───────X───@───────
              │
2: ───────────X───@───
                  │
3: ───────────────X───



In [10]:
# Convert Cirq circuit to squin kernel using bloqade
main_loaded = lowering_module.load_circuit(circuit, kernel_name="main")

print("\nSquin Kernel (from bloqade/cirq):")
print("=" * 60)
main_loaded.print()


Squin Kernel (from bloqade/cirq):
func.func @main() -> !py.NoneType {
  ^0(%main_self):
  │  %0 = py.constant.constant 4 : !py.int
  │  %1 = func.invoke qalloc(%0) : !py.IList[!py.Qubit, !Any] maybe_pure=False
  │  %2 = py.constant.constant 0 : !py.int
  │  %3 = py.indexing.getitem(%1, %2) : !py.Qubit
  │  %4 = py.ilist.new(values=(%3)){elem_type=!Any} : !py.IList[!py.Qubit, Literal(1,int)]
  │       squin.gate.h(qubits=%4)
  │  %5 = py.constant.constant 0 : !py.int
  │  %6 = py.indexing.getitem(%1, %5) : !py.Qubit
  │  %7 = py.ilist.new(values=(%6)){elem_type=!Any} : !py.IList[!py.Qubit, Literal(1,int)]
  │  %8 = py.constant.constant 1 : !py.int
  │  %9 = py.indexing.getitem(%1, %8) : !py.Qubit
  │ %10 = py.ilist.new(values=(%9)){elem_type=!Any} : !py.IList[!py.Qubit, Literal(1,int)]
  │       squin.gate.cx(controls=%7, targets=%10)
  │ %11 = py.constant.constant 1 : !py.int
  │ %12 = py.indexing.getitem(%1, %11) : !py.Qubit
  │ %13 = py.ilist.new(values=(%12)){elem_type=!Any} : !py.

## Run on Simulator and Compare output vectors

In [13]:
print("cirq->squin - output:")
cirq_sim = StackMemorySimulator(min_qubits=4)
cirq_result = cirq_sim.run(main_loaded)
print(cirq_sim.state_vector(main_loaded))

print("\n")

print("pyqir->squin - output:")
pyq_sim = StackMemorySimulator(min_qubits=4)
pyq_result = pyq_sim.run(squin_mod)
print(pyq_sim.state_vector(squin_mod))

cirq->squin - output:
[(0.7002180218696594-0.0984618067741394j), 0j, (-0+0j), 0j, 0j, (-0+0j), 0j, 0j, 0j, 0j, (-0+0j), 0j, 0j, (-0+0j), 0j, (0.7002180218696594-0.0984618067741394j)]


pyqir->squin - output:
[(0.707064688205719-0.007712312042713165j), -0j, -0j, -0j, 0j, 0j, 0j, (-0+0j), (-0+0j), 0j, 0j, 0j, 0j, 0j, 0j, (0.707064688205719-0.007712312042713165j)]


## Actual GHZ Squin Code

In [12]:
from bloqade import squin
from bloqade.pyqrack import StackMemorySimulator


@squin.kernel
def ghz(n: int):
    q = squin.qalloc(n)
    squin.h(q[0])
    for i in range(1, n):
        squin.cx(q[i - 1], q[i])

In [9]:
sim = StackMemorySimulator(min_qubits=4)
res = sim.run(ghz, args=(4,))  # need to pass in function arguments separately
sim.state_vector(ghz, args=(4,))

[(0.5533518195152283-0.4402291476726532j),
 (-0+0j),
 -0j,
 -0j,
 0j,
 0j,
 0j,
 (-0+0j),
 (-0+0j),
 0j,
 0j,
 0j,
 0j,
 0j,
 0j,
 (0.5533518195152283-0.4402291476726532j)]